In [ ]:
import pandas as pd
import numpy as np
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import os
import csv
import pickle

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn import preprocessing
from autokeras.image.image_supervised import load_image_dataset

from autokeras.image.image_supervised import ImageClassifier
from autokeras.image.image_supervised import load_image_dataset
import keras
import numpy as np
from keras.applications import vgg16, inception_v3, resnet50, mobilenet
 


from IPython.display import display
pd.set_option('display.max_rows',5)
pd.set_option('precision',3)

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important;}</style>")

In [ ]:
all_train_images=os.listdir("./data/box/images")
#print(len(all_train_images))
labels=[x.split("_")[0] for x in all_train_images]
labels_my=[int(x.split("_")[1].split('.png')[0]) for x in all_train_images]
labels_my

# trail all, delete

filt=pd.DataFrame({'image':all_train_images,'label':labels,'num':labels_my})
filt
all_train_images=filt['image'].tolist()
labels=filt['label'].tolist()

with open('train_label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(all_train_images)):
        writer.writerow({'File Name': all_train_images[i], 'Label':labels[i]})
    train_csv.close()

x_train, y_train = load_image_dataset(csv_file_path="train_label.csv",
                                      images_path="./data/box/images")
print(x_train.shape)
print(y_train.shape)


filt = pd.read_csv('./val/val_annotations.txt', sep="\t", header=None)
filt.columns=['image','label','a','b','c','d']
filt['image']=filt['image'].apply(lambda x: x.split('.')[0]+'.png')

all_train_images=filt['image'].tolist()
labels=filt['label'].tolist()

with open('valid_label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(all_train_images)):
        writer.writerow({'File Name': all_train_images[i], 'Label':labels[i]})
    train_csv.close()
    
x_valid, y_valid = load_image_dataset(csv_file_path="valid_label.csv",
                                      images_path="./data/val/box/images")
print(x_valid.shape)
print(y_valid.shape)


all_train_images=os.listdir("./data/test/box/images")
labels=['unknow' for x in all_train_images]
labels_my=['unknow' for x in all_train_images]


filt=pd.DataFrame({'image':all_train_images,'label':labels,'num':labels_my})
all_train_images=filt['image'].tolist()
labels=filt['label'].tolist()
import csv
with open('test_label.csv', 'w') as train_csv:
    fieldnames = ['File Name', 'Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(all_train_images)):
        writer.writerow({'File Name': all_train_images[i], 'Label':labels[i]})
    train_csv.close()
    
x_test, y_test = load_image_dataset(csv_file_path="test_label.csv",
                                      images_path="./data/test/box/images")
print(x_test.shape)


In [ ]:
x_train = x_train.astype("float") / 255.0
x_valid = x_valid.astype("float") / 255.0
x_test = x_test.astype("float") / 255.0

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train1=le.transform(y_train) 
y_train2= keras.utils.to_categorical(y_train1, 200)
y_valid1=le.transform(y_valid) 
y_valid2= keras.utils.to_categorical(y_valid1, 200)

In [ ]:
TRAINING_TIMES = [
    60 * 60 *0.1, # 6 hour
    60 * 60 * 2, # 2 hours
    60 * 60 * 3, # 3 hours
    60 * 60 * 4, # 4 hours
]

In [ ]:
results=[]
for seconds in TRAINING_TIMES:
    print("[INFO] training model for {} seconds max...".format(seconds))
    clf = ImageClassifier(verbose=True,augment=True, searcher_args={'trainer_args':{'max_no_improvement_num':4}})
    clf.fit(x_train, y_train, time_limit=seconds)
    fitted=clf.final_fit(x_train, y_train,x_valid, y_valid, retrain=True)
    print(clf.summary())
    # evaluate the Auto-Keras model
    score_train = clf.evaluate(x_train, y_train)
    score_valid = clf.evaluate(x_valid, y_valid)
    y_predict = clf.predict(x_test)
    
    print(score_train,score_valid)
    #results.append([clf.copy(),seconds,score_train,score_valid,score_test])
results

In [ ]:
# Score trained model.
scores = clf.evaluate(x_train, y_train, verbose=1)
print('Train loss:', scores[0])
print('Train accuracy:', scores[1])
scores = clf.evaluate(x_valid, y_valid, verbose=1)
print('Validation loss:', scores[0])
print('Validation accuracy:', scores[1])

In [ ]:
y_predict_p1=clf.predict(x_test)
y_predict_p1_1= [np.argmax(y, axis=None, out=None) for y in y_predict_p1]
y_predict_p1_2=le.inverse_transform(y_predict_p1_1)
y_predict_p1_2

In [ ]:
import csv
with open('y_predicted_p2.csv', 'w') as train_csv:
    fieldnames = ['Label']
    writer = csv.DictWriter(train_csv, fieldnames=fieldnames)
    writer.writeheader()
    for i in range(len(y_predict_p1_2)):
        writer.writerow({ 'Label':y_predict_p1_2[i]})
    train_csv.close()

In [ ]:
import matplotlib.pyplot as plt
history_dict=history.history
history_dict.keys()

In [ ]:
history_dict = fitted.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history_dict['acc']
val_acc = history_dict['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()